# Plotting and Data Access

This notebook provides a comprehensive guide to accessing optimization results and creating visualizations in flixopt.

**Topics covered:**
- Loading saved FlowSystems from NetCDF files
- Accessing data (flow rates, sizes, effects, charge states)
- Time series plots (balance, flows, storage)
- Aggregated plots (sizes, effects, duration curves)
- Heatmaps with time reshaping
- Sankey diagrams
- Topology visualization
- Color customization and export

## Setup

In [1]:
from pathlib import Path

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## Generate Example Data

First, run the script that generates three example FlowSystems with solutions:

In [2]:
# Run the generation script (only needed once, or to regenerate)
!python data/generate_example_systems.py

Creating simple_system...
  Optimizing...
  Saving to /Users/felix/PycharmProjects/flixopt_182303/docs/notebooks/data/simple_system.nc4...
  Done. Objective: 558.83

Creating complex_system...
  Optimizing...
  Saving to /Users/felix/PycharmProjects/flixopt_182303/docs/notebooks/data/complex_system.nc4...
  Done. Objective: 220.25

Creating multiperiod_system...
  Optimizing...
  Saving to /Users/felix/PycharmProjects/flixopt_182303/docs/notebooks/data/multiperiod_system.nc4...
  Done. Objective: 644.93

All systems generated successfully!


## 1. Loading Saved FlowSystems

FlowSystems can be saved to and loaded from NetCDF files, preserving the full structure and solution:

In [3]:
DATA_DIR = Path('data')

# Load the three example systems
simple = fx.FlowSystem.from_netcdf(DATA_DIR / 'simple_system.nc4')
complex_sys = fx.FlowSystem.from_netcdf(DATA_DIR / 'complex_system.nc4')
multiperiod = fx.FlowSystem.from_netcdf(DATA_DIR / 'multiperiod_system.nc4')

print('Loaded systems:')
print(f'  simple:      {len(simple.components)} components, {len(simple.buses)} buses')
print(f'  complex_sys: {len(complex_sys.components)} components, {len(complex_sys.buses)} buses')
print(f'  multiperiod: {len(multiperiod.components)} components, dims={dict(multiperiod.solution.sizes)}')

Loaded systems:
  simple:      4 components, 2 buses
  complex_sys: 9 components, 3 buses
  multiperiod: 4 components, dims={'scenario': 2, 'period': 3, 'time': 49}


## 2. Data Access via `statistics`

The `statistics` accessor provides convenient access to processed optimization results.

### 2.1 Flow Rates

Time series of flow rates (power) for each flow:

In [4]:
flow_rates = simple.statistics.flow_rates
print('Flow rates dataset:')
print(f'  Variables: {list(flow_rates.data_vars)}')
print(f'  Dimensions: {dict(flow_rates.sizes)}')
flow_rates

Flow rates dataset:
  Variables: ['GasGrid(Gas)', 'Boiler(Gas)', 'Boiler(Heat)', 'ThermalStorage(Charge)', 'ThermalStorage(Discharge)', 'Office(Heat)']
  Dimensions: {'time': 169}


<xarray.Dataset> Size: 9kB
Dimensions:                    (time: 169)
Coordinates:
  * time                       (time) datetime64[ns] 1kB 2024-01-15 ... 2024-...
Data variables:
    GasGrid(Gas)               (time) float64 1kB 35.31 31.86 ... 135.3 nan
    Boiler(Gas)                (time) float64 1kB 35.31 31.86 ... 135.3 nan
    Boiler(Heat)               (time) float64 1kB 32.48 29.31 ... 124.5 nan
    ThermalStorage(Charge)     (time) float64 1kB 0.0 -3.748e-13 ... 100.0 nan
    ThermalStorage(Discharge)  (time) float64 1kB 0.0 -5.275e-13 ... nan
    Office(Heat)               (time) float64 1kB 32.48 29.31 ... 24.48 nan

### 2.2 Flow Hours (Energy)

Energy values (flow rate × time step duration):

In [5]:
flow_hours = simple.statistics.flow_hours
print('Total energy by flow:')
for var in flow_hours.data_vars:
    total = flow_hours[var].sum().item()
    print(f'  {var}: {total:.1f} kWh')

Total energy by flow:
  GasGrid(Gas): 8936.7 kWh
  Boiler(Gas): 8936.7 kWh
  Boiler(Heat): 8221.7 kWh
  ThermalStorage(Charge): 3457.2 kWh
  ThermalStorage(Discharge): 3242.8 kWh
  Office(Heat): 8007.3 kWh


### 2.3 Sizes

Component and flow sizes (capacity values):

In [6]:
sizes = simple.statistics.sizes
print('Sizes:')
sizes

Sizes:


<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*

### 2.4 Charge States (Storage)

Time series of storage charge levels:

In [7]:
charge_states = simple.statistics.charge_states
print('Charge states:')
charge_states

Charge states:


<xarray.Dataset> Size: 3kB
Dimensions:         (time: 169)
Coordinates:
  * time            (time) datetime64[ns] 1kB 2024-01-15 ... 2024-01-22
Data variables:
    ThermalStorage  (time) float64 1kB 250.0 248.8 247.5 ... 103.0 102.5 200.0

### 2.5 Effects

Effects (costs, emissions, etc.) at different aggregation levels:

In [8]:
# Temporal effects (per timestep)
print('Temporal effects (time series):')
print(simple.statistics.temporal_effects)

# Total effects (scalar)
print('\nTotal effects:')
print(simple.statistics.total_effects)

Temporal effects (time series):
<xarray.Dataset> Size: 4kB
Dimensions:         (contributor: 1, time: 169)
Coordinates:
  * contributor     (contributor) object 8B 'GasGrid(Gas)'
  * time            (time) datetime64[ns] 1kB 2024-01-15 ... 2024-01-22
    component       (contributor) <U7 28B 'GasGrid'
    component_type  (contributor) <U6 24B 'Source'
Data variables:
    costs           (time, contributor) float64 1kB 1.765 1.593 ... 6.765 nan
    Penalty         (time, contributor) float64 1kB nan nan nan ... nan nan nan

Total effects:
<xarray.Dataset> Size: 76B
Dimensions:         (contributor: 1)
Coordinates:
  * contributor     (contributor) object 8B 'GasGrid(Gas)'
    component       (contributor) <U7 28B 'GasGrid'
    component_type  (contributor) <U6 24B 'Source'
Data variables:
    costs           (contributor) float64 8B 558.8
    Penalty         (contributor) float64 8B nan


For multi-effect systems:

In [9]:
print('Complex system effects:')
print(f'  Effects defined: {list(complex_sys.effects.keys())}')
print('\nTotal effects:')
complex_sys.statistics.total_effects

Complex system effects:
  Effects defined: ['costs', 'CO2', 'Penalty']

Total effects:


<xarray.Dataset> Size: 640B
Dimensions:         (contributor: 4)
Coordinates:
  * contributor     (contributor) object 32B 'CHP' ... 'GasGrid(Gas)'
    component       (contributor) <U17 272B 'CHP' ... 'GasGrid'
    component_type  (contributor) <U15 240B 'LinearConverter' ... 'Source'
Data variables:
    costs           (contributor) float64 32B 78.0 -386.3 118.1 410.4
    CO2             (contributor) float64 32B nan nan 295.3 1.368e+03
    Penalty         (contributor) float64 32B nan nan nan nan

### 2.6 Cached Colors

Colors are cached for consistent visualization:

In [10]:
print('Carrier colors:', simple.topology.carrier_colors)
print('Bus colors:', simple.topology.bus_colors)
print('Component colors:', simple.topology.component_colors)

Carrier colors: {'gas': '#1F77B4', 'heat': '#D62728'}
Bus colors: {'Gas': '#1F77B4', 'Heat': '#D62728'}
Component colors: {}


## 3. Time Series Plots

### 3.1 Balance Plot

Shows inflows and outflows for a bus or component:

In [11]:
# Bus balance (all flows connected to a bus)
simple.statistics.plot.balance('Heat')

2025-12-12 12:56:23.681 WARNING  │ No flows remaining after filtering for node Heat


PlotResult(data=<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*, figure=Figure({
    'data': [], 'layout': {'template': '...'}
}))

In [12]:
# Component balance (all flows of a component)
simple.statistics.plot.balance('ThermalStorage')

PlotResult(data=<xarray.Dataset> Size: 4kB
Dimensions:                    (time: 169)
Coordinates:
  * time                       (time) datetime64[ns] 1kB 2024-01-15 ... 2024-...
Data variables:
    ThermalStorage(Charge)     (time) float64 1kB -0.0 3.748e-13 ... -100.0 nan
    ThermalStorage(Discharge)  (time) float64 1kB 0.0 -5.275e-13 ... nan, figure=Figure({
    'data': [{'hovertemplate': 'variable=ThermalStorage(Charge)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'ThermalStorage(Charge)',
              'marker': {'color': '#D62728', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'ThermalStorage(Charge)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2024-01-15T00:00:00.000000000', '2024-01-15T01:00:00.000000000',
                          '2024-01-15T02:00:00.000000000', '2024-01-15T03:00:00.000000000',
                          '2024-01-15T04:00:00.000000000', '2024-01-15T05:00:00.000000000',
                          '2024-01-15T06:00:00.000000000', '2024-01-15T07:00:00.000000000',
                          '2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000', '2024-01-15T13:00:00.000000000',
                          '2024-01-15T14:00:00.000000000', '2024-01-15T15:00:00.000000000',
                          '2024-01-15T16:00:00.000000000', '2024-01-15T17:00:00.000000000',
                          '2024-01-15T18:00:00.000000000', '2024-01-15T19:00:00.000000000',
                          '2024-01-15T20:00:00.000000000', '2024-01-15T21:00:00.000000000',
                          '2024-01-15T22:00:00.000000000', '2024-01-15T23:00:00.000000000',
                          '2024-01-16T00:00:00.000000000', '2024-01-16T01:00:00.000000000',
                          '2024-01-16T02:00:00.000000000', '2024-01-16T03:00:00.000000000',
                          '2024-01-16T04:00:00.000000000', '2024-01-16T05:00:00.000000000',
                          '2024-01-16T06:00:00.000000000', '2024-01-16T07:00:00.000000000',
                          '2024-01-16T08:00:00.000000000', '2024-01-16T09:00:00.000000000',
                          '2024-01-16T10:00:00.000000000', '2024-01-16T11:00:00.000000000',
                          '2024-01-16T12:00:00.000000000', '2024-01-16T13:00:00.000000000',
                          '2024-01-16T14:00:00.000000000', '2024-01-16T15:00:00.000000000',
                          '2024-01-16T16:00:00.000000000', '2024-01-16T17:00:00.000000000',
                          '2024-01-16T18:00:00.000000000', '2024-01-16T19:00:00.000000000',
                          '2024-01-16T20:00:00.000000000', '2024-01-16T21:00:00.000000000',
                          '2024-01-16T22:00:00.000000000', '2024-01-16T23:00:00.000000000',
                          '2024-01-17T00:00:00.000000000', '2024-01-17T01:00:00.000000000',
                          '2024-01-17T02:00:00.000000000', '2024-01-17T03:00:00.000000000',
                          '2024-01-17T04:00:00.000000000', '2024-01-17T05:00:00.000000000',
                          '2024-01-17T06:00:00.000000000', '2024-01-17T07:00:00.000000000',
                          '2024-01-17T08:00:00.000000000', '2024-01-17T09:00:00.000000000',
                          '2024-01-17T10:00:00.000000000', '2024-01-17T11:00:00.000000000',
                          '2024-01-17T12:00:00.000000000', '2024-01-17T13:00:00.000000000',
                          '2024-01-17T14:00:00.000000000', '2024-01-17T15:00:00.000000000',
                          '2024-01-17T16:00:00.000000000', '2024-01-17T17:00:00.000000000',
                          '2024-01-17T18:00:00.000000000', '2024-01-17T19:00:00.000000000',
                          '2024-01-17T20:00:00.000000000', '2024-0

### 3.2 Carrier Balance

Shows all flows of a specific carrier across the system:

In [13]:
complex_sys.statistics.plot.carrier_balance('heat')

2025-12-12 12:56:23.853 WARNING  │ No flows remaining after filtering for carrier heat


PlotResult(data=<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*, figure=Figure({
    'data': [], 'layout': {'template': '...'}
}))

In [14]:
complex_sys.statistics.plot.carrier_balance('electricity')

2025-12-12 12:56:23.893 WARNING  │ No flows remaining after filtering for carrier electricity


PlotResult(data=<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*, figure=Figure({
    'data': [], 'layout': {'template': '...'}
}))

### 3.3 Flow Rates Plot

Plot multiple flow rates together:

In [15]:
# All flows
simple.statistics.plot.flows()

PlotResult(data=<xarray.Dataset> Size: 9kB
Dimensions:                    (time: 169)
Coordinates:
  * time                       (time) datetime64[ns] 1kB 2024-01-15 ... 2024-...
Data variables:
    GasGrid(Gas)               (time) float64 1kB 35.31 31.86 ... 135.3 nan
    Boiler(Gas)                (time) float64 1kB 35.31 31.86 ... 135.3 nan
    Boiler(Heat)               (time) float64 1kB 32.48 29.31 ... 124.5 nan
    ThermalStorage(Charge)     (time) float64 1kB 0.0 -3.748e-13 ... 100.0 nan
    ThermalStorage(Discharge)  (time) float64 1kB 0.0 -5.275e-13 ... nan
    Office(Heat)               (time) float64 1kB 32.48 29.31 ... 24.48 nan, figure=Figure({
    'data': [{'hovertemplate': 'variable=GasGrid(Gas)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'GasGrid(Gas)',
              'line': {'color': '#636EFA', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'GasGrid(Gas)',
              'showlegend': True,
              'type': 'scattergl',
              'x': array(['2024-01-15T00:00:00.000000000', '2024-01-15T01:00:00.000000000',
                          '2024-01-15T02:00:00.000000000', '2024-01-15T03:00:00.000000000',
                          '2024-01-15T04:00:00.000000000', '2024-01-15T05:00:00.000000000',
                          '2024-01-15T06:00:00.000000000', '2024-01-15T07:00:00.000000000',
                          '2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000', '2024-01-15T13:00:00.000000000',
                          '2024-01-15T14:00:00.000000000', '2024-01-15T15:00:00.000000000',
                          '2024-01-15T16:00:00.000000000', '2024-01-15T17:00:00.000000000',
                          '2024-01-15T18:00:00.000000000', '2024-01-15T19:00:00.000000000',
                          '2024-01-15T20:00:00.000000000', '2024-01-15T21:00:00.000000000',
                          '2024-01-15T22:00:00.000000000', '2024-01-15T23:00:00.000000000',
                          '2024-01-16T00:00:00.000000000', '2024-01-16T01:00:00.000000000',
                          '2024-01-16T02:00:00.000000000', '2024-01-16T03:00:00.000000000',
                          '2024-01-16T04:00:00.000000000', '2024-01-16T05:00:00.000000000',
                          '2024-01-16T06:00:00.000000000', '2024-01-16T07:00:00.000000000',
                          '2024-01-16T08:00:00.000000000', '2024-01-16T09:00:00.000000000',
                          '2024-01-16T10:00:00.000000000', '2024-01-16T11:00:00.000000000',
                          '2024-01-16T12:00:00.000000000', '2024-01-16T13:00:00.000000000',
                          '2024-01-16T14:00:00.000000000', '2024-01-16T15:00:00.000000000',
                          '2024-01-16T16:00:00.000000000', '2024-01-16T17:00:00.000000000',
                          '2024-01-16T18:00:00.000000000', '2024-01-16T19:00:00.000000000',
                          '2024-01-16T20:00:00.000000000', '2024-01-16T21:00:00.000000000',
                          '2024-01-16T22:00:00.000000000', '2024-01-16T23:00:00.000000000',
                          '2024-01-17T00:00:00.000000000', '2024-01-17T01:00:00.000000000',
                          '2024-01-17T02:00:00.000000000', '2024-01-17T03:00:00.000000000',
                          '2024-01-17T04:00:00.000000000', '2024-01-17T05:00:00.000000000',
                          '2024-01-17T06:00:00.000000000', '2024-01-17T07:00:00.000000000',
                          '2024-01-17T08:00:00.000000000', '2024-01-17T09:00:00.000000000',
                          '2024-01-17T10:00:00.000000000', '2024-01-17T11:00:00.000000000',
                          '2024-01-17T12:00:00.000000000', '2024-01-17T13:00:00.000000000',
                          '2024-01-17T14:00:00.000000000', '2024-01-17T15:00:00.000000000',

In [17]:
# Specific flows with line mode
simple.statistics.plot.flows(variables=['Boiler(Heat)', 'Office(Heat)'], mode='line')

TypeError: line() got an unexpected keyword argument 'variables'

### 3.4 Storage Plot

Combined view of storage charge state and flows:

In [ ]:
simple.statistics.plot.storage('ThermalStorage')

### 3.5 Charge States Plot

Plot charge state time series directly:

In [ ]:
simple.statistics.plot.charge_states('ThermalStorage')

## 4. Aggregated Plots

### 4.1 Sizes Plot

Bar chart of component/flow sizes:

In [ ]:
simple.statistics.plot.sizes()

### 4.2 Effects Plot

Bar chart of effect totals by component:

In [ ]:
simple.statistics.plot.effects('costs')

In [ ]:
# Multi-effect system: compare costs and CO2
complex_sys.statistics.plot.effects('costs', title='Operating Costs by Component')

In [ ]:
complex_sys.statistics.plot.effects('CO2', title='CO2 Emissions by Component')

### 4.3 Duration Curve

Shows how often each power level is reached:

In [ ]:
simple.statistics.plot.duration_curve('Boiler(Heat)')

In [ ]:
# Multiple variables
complex_sys.statistics.plot.duration_curve(
    ['CHP(Heat)', 'HeatPump(Heat)', 'BackupBoiler(Q_th)'], title='Heat Generation Duration Curves'
)

## 5. Heatmaps

Heatmaps reshape time series into 2D grids (e.g., hour-of-day vs day):

In [ ]:
# Auto-reshape based on data frequency
simple.statistics.plot.heatmap('Boiler(Heat)')

In [ ]:
# Storage charge state heatmap
simple.statistics.plot.heatmap('ThermalStorage')

In [ ]:
# Custom colorscale
simple.statistics.plot.heatmap('Office(Heat)', color_continuous_scale='Blues', title='Heat Demand Pattern')

## 6. Sankey Diagrams

Sankey diagrams visualize energy flows through the system.

### 6.1 Flow Sankey

Total energy flows:

In [ ]:
simple.statistics.plot.sankey.flows()

In [ ]:
# Complex system with multiple carriers
complex_sys.statistics.plot.sankey.flows()

### 6.2 Sizes Sankey

Capacity/size allocation:

In [ ]:
simple.statistics.plot.sankey.sizes()

### 6.3 Peak Flow Sankey

Maximum flow rates (peak power):

In [ ]:
simple.statistics.plot.sankey.peak_flow()

### 6.4 Effects Sankey

Cost/emission allocation:

In [ ]:
simple.statistics.plot.sankey.effects('costs')

In [ ]:
# CO2 allocation in complex system
complex_sys.statistics.plot.sankey.effects('CO2', title='CO2 Emissions Flow')

### 6.5 Filtering with `select`

Filter Sankey to specific buses or carriers:

In [ ]:
# Only heat flows
complex_sys.statistics.plot.sankey.flows(select={'bus': 'Heat'})

## 7. Topology Visualization

Visualize the system structure (no solution data required).

### 7.1 Topology Plot

Sankey-style network diagram:

In [ ]:
simple.topology.plot()

In [ ]:
complex_sys.topology.plot(title='Complex System Topology')

### 7.2 Topology Info

Get node and edge information programmatically:

In [ ]:
nodes, edges = simple.topology.infos()

print('Nodes:')
for label, info in nodes.items():
    print(f'  {label}: {info["class"]}')

print('\nEdges (flows):')
for label, info in edges.items():
    print(f'  {info["start"]} -> {info["end"]}: {label}')

## 8. Multi-Period/Scenario Data

Working with multi-dimensional results:

In [ ]:
print('Multiperiod system dimensions:')
print(f'  Periods: {list(multiperiod.periods)}')
print(f'  Scenarios: {list(multiperiod.scenarios)}')
print(f'  Solution dims: {dict(multiperiod.solution.sizes)}')

In [ ]:
# Balance plot with faceting by scenario
multiperiod.statistics.plot.balance('Heat')

In [ ]:
# Filter to specific scenario/period
multiperiod.statistics.plot.balance('Heat', select={'scenario': 'high_demand', 'period': 2024})

In [ ]:
# Sankey aggregates across all dimensions by default
multiperiod.statistics.plot.sankey.flows()

## 9. Color Customization

Colors can be customized in multiple ways:

In [ ]:
# Using a colorscale name
simple.statistics.plot.balance('Heat', colors='Set2')

In [ ]:
# Using a list of colors
simple.statistics.plot.balance('Heat', colors=['#e41a1c', '#377eb8', '#4daf4a', '#984ea3'])

In [ ]:
# Using a dictionary for specific labels
simple.statistics.plot.balance(
    'Heat',
    colors={
        'Boiler(Heat)': 'orangered',
        'ThermalStorage(Charge)': 'steelblue',
        'ThermalStorage(Discharge)': 'lightblue',
        'Office(Heat)': 'forestgreen',
    },
)

## 10. Exporting Results

Plots return a `PlotResult` with data and figure that can be exported:

In [ ]:
# Get plot result
result = simple.statistics.plot.balance('Heat')

print('PlotResult contains:')
print(f'  data: {type(result.data).__name__} with vars {list(result.data.data_vars)}')
print(f'  figure: {type(result.figure).__name__}')

In [ ]:
# Export data to pandas DataFrame
df = result.data.to_dataframe()
df.head()

In [ ]:
# Export figure to HTML (interactive)
# result.figure.write_html('balance_plot.html')

# Export figure to image
# result.figure.write_image('balance_plot.png', scale=2)

## 11. Using Matplotlib Backend

Most plots support both Plotly (default) and Matplotlib:

In [ ]:
# Balance with matplotlib
simple.statistics.plot.balance('Heat', backend='matplotlib')

In [ ]:
# Heatmap with matplotlib
simple.statistics.plot.heatmap('Boiler(Heat)', backend='matplotlib')

## Summary

### Data Access

| Property | Description |
|----------|-------------|
| `statistics.flow_rates` | Time series of flow rates (power) |
| `statistics.flow_hours` | Energy values (rate × duration) |
| `statistics.sizes` | Component/flow capacities |
| `statistics.charge_states` | Storage charge levels |
| `statistics.temporal_effects` | Effects per timestep |
| `statistics.periodic_effects` | Effects per period |
| `statistics.total_effects` | Aggregated effect totals |
| `topology.carrier_colors` | Cached carrier color mapping |
| `topology.component_colors` | Cached component color mapping |
| `topology.bus_colors` | Cached bus color mapping |

### Plot Methods

| Method | Description |
|--------|-------------|
| `plot.balance(node)` | Stacked bar of in/outflows |
| `plot.carrier_balance(carrier)` | Balance for all flows of a carrier |
| `plot.flows(variables)` | Time series line/area plot |
| `plot.storage(component)` | Combined charge state and flows |
| `plot.charge_states(component)` | Charge state time series |
| `plot.sizes()` | Bar chart of sizes |
| `plot.effects(effect)` | Bar chart of effect contributions |
| `plot.duration_curve(variables)` | Sorted duration curve |
| `plot.heatmap(variable)` | 2D time-reshaped heatmap |
| `plot.sankey.flows()` | Energy flow Sankey |
| `plot.sankey.sizes()` | Capacity Sankey |
| `plot.sankey.peak_flow()` | Peak power Sankey |
| `plot.sankey.effects(effect)` | Effect allocation Sankey |
| `topology.plot()` | System structure diagram |